In [1]:
import os
from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.openai_chat import OpenAIChat
from transformers import AutoTokenizer
from lm_polygraph.model_adapters import WhiteBoxModelBasic
from torch.utils.data import DataLoader
import json
import pandas as pd
from tqdm import tqdm
## Create a ClaimsExtractor object
calc_claim_extractor = ClaimsExtractor(OpenAIChat("gpt-4o"))
## Set model to None, as we will not need it 
model = None
tokenizer = AutoTokenizer.from_pretrained("core42/jais-13b", trust_remote_code=True)
## Setting a dummy whitebox model object
model_adapter = WhiteBoxModelBasic(model, tokenizer)


/home/boda/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree

In [2]:
## Load Arabic bios dataset
arabic_bios = []
with open("../workdir/arabian_output.json", "r") as f:
    arabic_bios = json.load(f) 

print(arabic_bios.keys())
print("Number of bios: ", len(arabic_bios['all_input_texts']))
arabic_bios = pd.DataFrame(arabic_bios, columns=['all_input_texts', 'all_output_text']).drop_duplicates()
print("Number of bios after removing duplicates: ", len(arabic_bios))

dict_keys(['all_input_texts', 'all_input_ids', 'all_generated_tokens', 'all_cut_alternatives', 'all_alternatives', 'all_output_text'])
Number of bios:  100
Number of bios after removing duplicates:  100


In [3]:
arabic_bios = arabic_bios.dropna().rename(columns={'all_input_texts': 'question', 'all_output_text': 'bio'})

In [5]:

bios_with_claims = []
## Create a DataLoader object
for  index, row in tqdm(arabic_bios.iterrows(), total=arabic_bios.shape[0]):

    question = row['question']
    bio = row['bio']

    
    encoded = tokenizer([bio], padding=True, return_tensors="pt")
    deps = {"greedy_tokens": encoded.input_ids, "greedy_texts": [bio]}
    claims = calc_claim_extractor(deps, texts=[bio], model=model_adapter,language="ar")
    
    for x in claims['claims'][0]:
        bios_with_claims.append({'question': question, 'bio': bio, "claim": x.claim_text, "sentence": x.sentence, "tokens": x.aligned_tokens})


100%|██████████| 100/100 [09:58<00:00,  5.99s/it]


In [6]:
## Save the extracted claims

save_df = pd.DataFrame(bios_with_claims, )

save_df.to_csv("../workdir/arabic_claims.csv", index=False)

In [7]:
len(save_df)

478

In [11]:
save_df['question'].unique().shape

(97,)